In [1]:
import datasets
import transformers
import pandas as pd
from train_datasets import BPEDropoutTrainDataset

In [2]:
from tqdm import tqdm

In [3]:
import numpy as np

In [4]:
cache_dir = "./cache"

In [18]:
dataset_path = "masakhaner"
language = "swa"
# dataset_path = "conll2003"
# language = None

# subword regularization params:
bpe_dropout_p = 0.05
model_path = "Davlan/afro-xlmr-mini"

In [19]:
train_dataset = BPEDropoutTrainDataset(dataset_path, model_path, dataset_language=language, bpe_dropout_p=bpe_dropout_p, cache_dir=cache_dir, train=True)

Found cached dataset masakhaner (/atlas2/u/xiluo/temp/cache/masakhaner/swa/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
tags = train_dataset.dset['train'].features["ner_tags"].feature
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], id=None)

In [21]:
tokenizer = train_dataset.tokenizer

In [22]:
pd.DataFrame(
    [tokenizer.convert_ids_to_tokens(train_dataset[0]['input_ids']), train_dataset[0]['labels']],
    index=["tokens", "ner_tags"])

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
tokens,<s>,▁Wizara,▁ya,▁afya,▁ya,▁Tanzania,▁i,meri,pot,i,...,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>
ner_tags,-100,3,4,4,4,4,0,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100


In [23]:
num_to_samp = 5

In [24]:
train_input_ids = np.zeros((len(train_dataset), num_to_samp, 512), dtype=int)
train_labels = np.zeros((len(train_dataset), num_to_samp, 512), dtype=int)
for idx in tqdm(range(len(train_dataset))):

    for i in range(num_to_samp):
        data = train_dataset[idx]

        train_input_ids[idx][i] = data['input_ids']
        train_labels[idx][i] = data['labels']        

    if idx == 10:
        break

  0%|▋                                                                                                                                             | 10/2109 [00:08<30:01,  1.16it/s]


In [16]:
train_input_ids[0]

array([[   0, 3692,   11, ...,    1,    1,    1],
       [   0, 3692,   11, ...,    1,    1,    1],
       [   0,  341, 2477, ...,    1,    1,    1],
       [   0,  341, 2477, ...,    1,    1,    1],
       [   0, 3692,  344, ...,    1,    1,    1]])

In [ ]:
np.savez("./xho_test.npz", input_ids=train_input_ids, labels=train_labels)

In [ ]:
train = np.load("./xho_test.npz")